# Generating Metadata and Linking Similar Dataset

Combining everything and saving files with data which can be used by D3.js to create network diagrams.

In [ ]:
#| default_exp utils.generate_Metadata

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import database_compendium.utils.ONS_scraper_functions as osf
import database_compendium.utils.Nomis_scraper_functions as nsf
import database_compendium.utils.insolvency_stats_scrapers as iss
import database_compendium.utils.NHS_QualityOutcomes_scrapers as qos
import database_compendium.utils.police_data_scrapers as pds
import database_compendium.utils.column_matching as cm

import numpy as np
import requests
import pandas as pd
import math

In [ ]:
#| export
def createMetadata():

    """ ONS Data """
    
    print("Loading ONS...")
    
    # Takes over 70 mins to run
    titles, descriptions = osf.get_ONS_datasets_titles_descriptions()
    urls = osf.get_ONS_datasets_urls()
    long_desc = osf.get_ONS_long_description()
    
    latest_release = []
    cols = []
    col_data = []
    count = 0
    
    for url in urls:
        response = requests.get(url)
        try: 
            latest_release.append(response.json()['release_date'])
        except:
            latest_release.append(float('nan'))
    
        try:
            cols.append(osf.find_ONS_cols(url))
        except:
            cols.append('')
    
    
        try:
            col_data.append(osf.find_ONS_cols_and_unique_vals(url))
        except:
            col_data.append('')
    
        
        count +=1
    
    ONS_df = pd.DataFrame({'Title': titles, 'Description': descriptions, 
                           'Long_description': long_desc, 'Columns': cols, 
                           'Unique_parameters': col_data, 'Latest_release': latest_release})
    
    
    """ Nomis Data """
    
    print("Loading Nomis...")
    
    # Takes around 50 mins
    titles, descriptions, l_descriptions = nsf.get_nomis_datasets_titles_descriptions()
    latest_release = nsf.get_nomis_last_updated()
    cols = np.empty(len(titles))
    params = nsf.get_nomis_datasets_parameters()
    
    nomis_df = pd.DataFrame({'Title': titles, 'Description': descriptions, 
                             'Long_description': l_descriptions, 'Columns': cols, 
                             'Unique_parameters': params, 'Latest_release': latest_release})


    """ Monthly Insolvency Statistics """
    
    print("Loading insolvency stats")
    cols = []
    col_data = []
    insolvency_stats, long_desc = iss.get_insolvency_stats()
    titles = insolvency_stats.keys()
    
    # The descriptions on latest releases are all the same so making a repeated list
    description = [iss.get_mis_description()] * len(titles)
    latest_release = [iss.get_mis_last_updated()] * len(titles)
    long_desc = [long_desc] * len(titles)
    
    for title in titles:
        cols.append(list(insolvency_stats[title].columns))
        col_data.append(iss.get_insolvency_unique_column_vals(insolvency_stats[title]))
    
    insolvency_df = pd.DataFrame({'Title': titles, 'Description': description, 
                              'Long_description': long_desc, 'Columns': cols, 
                              'Unique_parameters': col_data, 'Latest_release': latest_release})

    
    """ Police Data - currently only for Bethnal green so is commented out """
    
    print("Loding police data...")
    url = 'https://nihr.opendatasoft.com/api/records/1.0/search/?dataset=westminster-parliamentary-constituencies&rows=650'
    response = requests.get(url)
    records = response.json()['records']

    constituency_coords = {}

    for record in records:
        coords = []
        name = record['fields']['pcon22nm'] # This will change as the dataset gets updated each year
        temp_coords = record['fields']['geo_shape']['coordinates']
    
        temp_coords = pds.flatten_list(temp_coords)
    
        offset = math.floor(len(temp_coords)/4)
        constituency_coords[name] = (str(round(temp_coords[0][1], 3)) + ',' + str(round(temp_coords[0][0], 3)) + ':')
        constituency_coords[name] += (str(round(temp_coords[offset][1], 3)) + ',' + str(round(temp_coords[offset][0], 3)) + ':')
        constituency_coords[name] += (str(round(temp_coords[offset*2][1], 3)) + ',' + str(round(temp_coords[offset*2][0], 3)) + ':')
        constituency_coords[name] += (str(round(temp_coords[offset*3][1], 3)) + ',' + str(round(temp_coords[offset*3][0], 3)))

    street_level_crimes, sl_last_updated = pds.get_street_level_crimes(constituency_coords['Bethnal Green and Bow'], '2023-03', 'poly')
    no_loc_crimes = pds.get_crimes_no_loc('metropolitan', '2023-03')
    stop_searches, ss_last_updated = pds.get_stop_searches(constituency_coords['Bethnal Green and Bow'], '2023-03', 'poly')
    searches_no_loc = pds.get_searches_no_loc('metropolitan', '2023-03')

    col_data = []
    col_data.append(pds.get_unique_col_vals(street_level_crimes))
    col_data.append(pds.get_unique_col_vals(no_loc_crimes))
    col_data.append(pds.get_unique_col_vals(stop_searches))
    col_data.append(pds.get_unique_col_vals(searches_no_loc))
    
    street_level_crimes = pd.json_normalize(street_level_crimes)
    street_level_crimes = street_level_crimes.drop(columns='persistent_id')
    
    no_loc_crimes = pd.json_normalize(no_loc_crimes)
    no_loc_crimes = no_loc_crimes.drop(columns='persistent_id')
    
    stop_searches = pd.json_normalize(stop_searches)
    
    searches_no_loc = pd.json_normalize(searches_no_loc)
    searches_no_loc = searches_no_loc.drop(columns='datetime')

    titles = ['Police Data - Street-level crimes', 'Police Data - Crimes with no location',
          'Police Data - Stop and searches by area', 'Police Data - Stop and searches with no location']
    descriptions = ['Crimes at street-level; either within a 1 mile radius of a single point, or within a custom area.',
                    'Returns a list of crimes that could not be mapped to a location.',
                    'Stop and searches at street-level; either within a 1 mile radius of a single point, or within a custom area.',
                    'Stop and searches that could not be mapped to a location.']
    latest_release = [sl_last_updated, sl_last_updated, ss_last_updated, ss_last_updated]
    
    f = open('data/police_long_descriptions.txt', 'r')
    long_desc = f.read().split('\n')
    
    cols = []
    for data in col_data:
        cols.append(list(data.keys()))

    police_df = pd.DataFrame({'Title': titles, 'Description': descriptions, 
                          'Long_description': long_desc, 'Columns': cols, 
                          'Unique_parameters': col_data, 'Latest_release': latest_release})


    """ NHS Quality and Outcomes """

    print("Loading NHS Quality and Outcomes...")
    NHS_quality_outcomes, long_description, latest_release = qos.get_NHS_qualityOutcomes()

    titles = []
    for title in NHS_quality_outcomes.keys():
        titles.append(title.split(':')[1]) # Removing the text before : which is 'Table n:'
    
    num_tables = len(titles)
    descriptions = [''] * num_tables
    long_descriptions = [long_description] * num_tables
    last_rel = [latest_release] * num_tables
    
    cols = []
    unique_params = []
    for dataset in NHS_quality_outcomes.keys():
        sheet = NHS_quality_outcomes[dataset]
        temp_cols, temp_unqParams = qos.get_qualityOutcomes_uniqueColumnValues(sheet)
        cols.append(temp_cols)
        unique_params.append(temp_unqParams)

    qualOutcomes_df = pd.DataFrame({'Title': titles, 'Description': descriptions, 
                                    'Long_description': long_descriptions, 'Columns': cols, 
                                    'Unique_parameters': unique_params, 'Latest_release': latest_release})

    
    # metadata_df = pd.concat([ONS_df, nomis_df, insolvency_df, police_df, ]).reset_index(drop=True)
    metadata_df = pd.concat([police_df, qualOutcomes_df]).reset_index(drop=True)
    print("Complete")

    return metadata_df

In [ ]:
metadata_df = createMetadata()

Loading ONS...
Loading Nomis...
Loading insolvency stats
Loding police data...
Loading NHS Quality and Outcomes...
Complete


In [ ]:
metadata_df

,Title,Description,Long_description,Columns,Unique_parameters,Latest_release
0,Police Data - Street-level crimes,Crimes at street-level; either within a 1 mile...,The Police API Documentation for Street-Level ...,"[category, location_type, context, location_su...","{'category': ['anti-social-behaviour', 'bicycl...",2023-06-01
1,Police Data - Crimes with no location,Returns a list of crimes that could not be map...,The Police API Documentation for Crimes with N...,"[category, context, location_subtype, outcome_...","{'category': ['bicycle-theft', 'burglary', 'cr...",2023-06-01
2,Police Data - Stop and searches by area,Stop and searches at street-level; either with...,The Police API Documentation for Stop and Sear...,"[outcome, self_defined_ethnicity, gender, legi...","{'outcome': ['A no further action disposal', '...",2023-06
3,Police Data - Stop and searches with no location,Stop and searches that could not be mapped to ...,The Police API Documentation for Stop and Sear...,"[age_range, outcome, self_defined_ethnicity, g...","{'age_range': ['over 34', '18-24', None, '25-3...",2023-06
4,"Prevalence, achievement and personalised care...",,The objective of the Quality and Outcomes Fra...,"[Region name, Achievement Score (max 12), Regi...","{'Region ODS code': ['ENG', 'Y56', 'Y58', 'Y59...",22/09/2022
5,Achievement and personalised care adjustments...,,The objective of the Quality and Outcomes Fra...,"[Region name, Region ODS code, Total Achieveme...","{'Region ODS code': ['ENG', 'Y56', 'Y58', 'Y59...",22/09/2022
6,"Prevalence, achievement and personalised care...",,The objective of the Quality and Outcomes Fra...,"[Region name, Achievement Score (max 12), Regi...","{'Region ODS code': ['ENG', 'Y56', 'Y58', 'Y59...",22/09/2022
7,"Prevalence, achievement and personalised care...",,The objective of the Quality and Outcomes Fra...,"[Region name, Achievement Score (max 12), Achi...","{'Region ODS code': ['ENG', 'Y56', 'Y58', 'Y59...",22/09/2022
8,"Prevalence, achievement and personalised care...",,The objective of the Quality and Outcomes Fra...,"[Region name, Region ODS code, Total Denominat...","{'Region ODS code': ['ENG', 'Y56', 'Y58', 'Y59...",22/09/2022
9,"Prevalence and achievement, cardiovascular gr...",,The objective of the Quality and Outcomes Fra...,"[Region name, List size, Region ONS code, Achi...","{'Region ODS code': ['ENG', 'Y56', 'Y58', 'Y59...",22/09/2022


In [ ]:
#| export
def prepare_identicalColData(metadata_df,         # Dataset
                             method="",           # Leave blank for the default or 'alt' for the alternative method
                             rare=True,           # Increases the weight given to rare connections
                             cutoff=5,            # The minimum weight of a connection to be included in the final file
                             filename="testing"): # The name given to the output file saved to the local data folder
    """ 
    Using the metadata this creates a dataset showing the connections between datasets 
    using the column data and formats it for use in the d3 network diagram. 
    Saves it to a file, filename and also returns the data as a dataframe.
    """
    cols_list = cm.createColsList(metadata_df)
    scoredConnections = cm.scoreConnections(metadata_df, cols_list, method=method, rare=True)
    iColData_d3 = cm.formatForD3(metadata_df.Title, scoredConnections, cutoff, True, filename) # True in this case just means the file is being saved which I'm assuming we want

    return iColData_d3

In [ ]:
metadata_df = pd.read_csv('~/database_compendium/data/metadata_w_nomis_description.csv')
prepare_identicalColData(metadata_df)

,Source,Target,weight
0,0,1,10.0
1,0,2,10.0
2,0,3,10.0
3,0,4,10.0
4,0,5,10.0
...,...,...,...
129681,1920,1923,10.0
129682,1922,1923,10.0
129683,1924,1925,10.0
129684,1924,1926,10.0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()